Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# must hit 90% accuracy, but only get 2 submissions

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pieplines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

In [1]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [8]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [21]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()

In [10]:
# Define the Pipeline
pipe = Pipeline([('vect', vect), ('clf', rfc)])

In [12]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (100, 500,1000),
    'clf__n_estimators':(50, 100),
    'clf__max_depth':(10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=2, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 2 folds for each of 72 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:  1.1min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (100, 500, 1000), 'clf__n_estimators': (50, 100), 'clf__max_depth': (10, 15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [13]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1], dtype=int64)

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [2]:
import pandas as pd

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
train.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


In [4]:
test.head()

,id,description
0,955,"Think carnival aromas—the good ones, anyway—me..."
1,3532,"A blend of three bourbons, between 6 and 12 ye..."
2,1390,"The nose is focused on cereal, hints of fresh ..."
3,1024,Swiss-based Chapter 7 released this 19 year ol...
4,1902,Valkyrie replaces the current Dark Origins exp...


In [4]:
X = train['description']
y = train['category']

X.shape, y.shape

((2586,), (2586,))

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2068,), (518,), (2068,), (518,))

### Define Pipeline Components

In [36]:
vect = TfidfVectorizer(stop_words='english')
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [37]:
param_dist = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (100, 500,1000),
    'clf__max_depth':(5,10,15,20, None),
    'clf__n_estimators': (100, 500, 1000)
}

search = RandomizedSearchCV(pipe, param_distributions=param_dist, cv=5, n_jobs=4, verbose=10,
                           n_iter=50, scoring='accuracy', random_state=42)
search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   10.9s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   13.7s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:   17.9s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   23.0s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   37.4s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   53.0s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  3.6min
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:  4.1min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  4.5min
[Parallel(n_jobs=4)]: Done 173 tasks      | elapsed:  5.5min
[Parallel(

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=50, n_jobs=4,
          param_distributions={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (100, 500, 1000), 'clf__max_depth': (5, 10, 15, 20, None), 'clf__n_estimators': (100, 500, 1000)},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=10)

In [38]:
search.best_estimator_

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.75, max_features=1000, min_df=0.02,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [39]:
search.best_score_

0.8999032882011605

In [40]:
from sklearn.metrics import accuracy_score

y_pred = search.predict(X_test)
score = accuracy_score(y_test, y_pred)
score

0.8822393822393823

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [41]:
# Predictions on test sample
pred = search.predict(test['description'])

In [42]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [43]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [44]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

## Challenge

You're trying to achienve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [6]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, 
                   algorithm='arpack')

In [17]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': [.9, .95, 1.0]
}

In [18]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [19]:
# Fit
# pipe.fit(data.data, data.target)
grid_search = GridSearchCV(pipe, params, cv=5, n_jobs=4, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:  2.8min finished
C:\Users\theta\Anaconda3\envs\u4_s1_nlp\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'lsi__svd__n_components': [10, 100, 250], 'lsi__vect__max_df': [0.9, 0.95, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [21]:
grid_search.best_score_

0.882147024504084

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [48]:
lsi = Pipeline([('vect', vect), ('svd', svd)])
vect = TfidfVectorizer(stop_words='english')
clf = RandomForestClassifier()

pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [69]:
parameters = {
    'lsi__svd__n_components': [10,99],
    'lsi__vect__max_df': (0.75, 1.0),
    'lsi__vect__min_df': (.02, .05),
    'lsi__vect__max_features': (100, 500, 1000),
    'clf__max_depth':(10,15,20, None),
    'clf__n_estimators': (100, 500, 1000)
}

search = RandomizedSearchCV(pipe, parameters, cv=5, n_jobs=4, verbose=10, n_iter=50, random_state=42)
search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   15.6s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   22.6s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:   41.8s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:  3.3min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:  3.8min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  5.1min
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:  6.5min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  8.3min
[Parallel(n_jobs=4)]: Done 173 tasks      | elapsed:  9.4min
[Parallel(

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=50, n_jobs=4,
          param_distributions={'lsi__svd__n_components': [10, 99], 'lsi__vect__max_df': (0.75, 1.0), 'lsi__vect__min_df': (0.02, 0.05), 'lsi__vect__max_features': (100, 500, 1000), 'clf__max_depth': (10, 15, 20, None), 'clf__n_estimators': (100, 500, 1000)},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=10)

In [71]:
search.best_estimator_

Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=500, min_df=0.02,
        ngram_range=(1, 1), nor...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [72]:
search.best_score_

0.8907156673114119

In [73]:
y_pred = search.predict(X_test)
score = accuracy_score(y_test, y_pred)

In [74]:
score

0.8783783783783784

# Model using SGDClassifier

In [10]:
svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized')

In [11]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))
lsi = Pipeline([('vect', vect), ('svd', svd)])
clf = SGDClassifier(early_stopping=True, random_state=42)

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [54]:
X_train.shape, y_train.shape

((2068,), (2068,))

In [12]:
X = train['description']
y = train['category']

X.shape, y.shape

((2586,), (2586,))

In [15]:
parameters = {
    'lsi__svd__n_components': [50,100,150],
    'clf__loss': ['hinge', 'log'],
    'clf__alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3],
    'clf__tol': [1e-4, 1e-3, 1e-2, 1e-1]
}

search = RandomizedSearchCV(pipe, parameters, cv=10, n_jobs=4, verbose=10)
search.fit(X, y)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   39.1s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   56.5s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  4.9min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  5.8min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  7.2min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  9.0min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed: 10.7min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed: 13.2min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed: 14.4min finished
C:\Users\theta\Anaconda3\envs\u4_s1_nlp\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm=...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=4,
          param_distributions={'lsi__svd__n_components': [50, 100, 150], 'clf__loss': ['hinge', 'log'], 'clf__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'clf__tol': [0.0001, 0.001, 0.01, 0.1]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=10)

In [16]:
search.best_estimator_

Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm=..._state=None, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [17]:
search.best_score_

0.9187935034802784

In [18]:
y_pred = search.predict(X_test)
score = accuracy_score(y_test, y_pred)

In [19]:
score

0.9343629343629344

# Model using LinearSVC

In [24]:
svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized')

In [25]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))
lsi = Pipeline([('vect', vect), ('svd', svd)])
clf = LinearSVC(random_state=42)

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [26]:
parameters = {
    'lsi__svd__n_components': [50,100,150],
    'clf__loss': ['hinge', 'squared_hinge'],
    'clf__tol': [1e-4, 1e-3, 1e-2, 1e-1]
}

grid_search = GridSearchCV(pipe, parameters, cv=10, n_jobs=4, verbose=10)
grid_search.fit(X, y)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   39.2s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   56.5s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  3.0min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  4.4min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  5.3min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  7.2min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  8.6min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed: 10.0min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed: 12.6min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed: 14.3min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 16.7min
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed: 18.4min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 21.0min
[Parallel(n_jobs=4)]: Done 173 tasks      | elapsed: 23.6min
[Parallel(

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm=... max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'lsi__svd__n_components': [50, 100, 150], 'clf__loss': ['hinge', 'squared_hinge'], 'clf__tol': [0.0001, 0.001, 0.01, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [27]:
grid_search.best_estimator_

Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm=..._hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.1, verbose=0))])

In [28]:
grid_search.best_score_

0.9276875483372004

In [29]:
y_pred = grid_search.predict(X_test)
score = accuracy_score(y_test, y_pred)
score

0.9555984555984556

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [30]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [31]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [32]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,3
2,1390,4
3,1024,1
4,1902,1


In [33]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission7.csv', index=False)

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [25]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [22]:
doc = nlp("Two bananas in pyjamas")

In [23]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [23]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [25]:
X = get_word_vectors(data.data)

rfc.fit(X, data.target)

C:\Users\theta\Anaconda3\envs\u4_s1_nlp\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [26]:
rfc.score(X, data.target)

0.9906651108518086

## Follow Along

In [61]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [28]:
X_train = get_word_vectors(X_train)
X_test = get_word_vectors(X_test)

In [29]:
model = RandomForestClassifier(n_estimators=1000, max_depth=None, random_state=42, verbose=10, n_jobs=4)
model.fit(X, y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 1000building tree 2 of 1000building tree 3 of 1000building tree 4 of 1000



building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000

[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.3s


building tree 10 of 1000
building tree 11 of 1000

building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000

[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.5s


...

building tree 921 of 1000
building tree 922 of 1000
building tree 923 of 1000
building tree 924 of 1000
building tree 925 of 1000building tree 926 of 1000

building tree 927 of 1000
building tree 928 of 1000
building tree 929 of 1000
building tree 930 of 1000
building tree 931 of 1000
building tree 932 of 1000building tree 933 of 1000

building tree 934 of 1000
building tree 935 of 1000
building tree 936 of 1000
building tree 937 of 1000
building tree 938 of 1000
building tree 939 of 1000
building tree 940 of 1000
building tree 941 of 1000
building tree 942 of 1000
building tree 943 of 1000
building tree 944 of 1000
building tree 945 of 1000
building tree 946 of 1000
building tree 947 of 1000
building tree 948 of 1000building tree 949 of 1000

building tree 950 of 1000
building tree 951 of 1000
building tree 952 of 1000building tree 953 of 1000

building tree 954 of 1000
building tree 955 of 1000
building tree 956 of 1000
building tree 957 of 1000
building tree 958 of 1000
building tre

[Parallel(n_jobs=4)]: Done 960 tasks      | elapsed:   35.6s



building tree 965 of 1000
building tree 966 of 1000building tree 967 of 1000

building tree 968 of 1000
building tree 969 of 1000
building tree 970 of 1000
building tree 971 of 1000
building tree 972 of 1000
building tree 973 of 1000building tree 974 of 1000

building tree 975 of 1000
building tree 976 of 1000
building tree 977 of 1000
building tree 978 of 1000
building tree 979 of 1000
building tree 980 of 1000
building tree 981 of 1000
building tree 982 of 1000building tree 983 of 1000

building tree 984 of 1000
building tree 985 of 1000
building tree 986 of 1000
building tree 987 of 1000
building tree 988 of 1000
building tree 989 of 1000
building tree 990 of 1000
building tree 991 of 1000building tree 992 of 1000

building tree 993 of 1000
building tree 994 of 1000
building tree 995 of 1000
building tree 996 of 1000
building tree 997 of 1000
building tree 998 of 1000
building tree 999 of 1000building tree 1000 of 1000



[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:   37.1s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=4,
            oob_score=False, random_state=42, verbose=10, warm_start=False)

In [30]:
y_pred = model.predict(X_test)
score = accuracy_score(y_test, y_pred)
score

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 173 tasks      | elapsed:    0.2s
[Para

0.7162162162162162

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on